<h1 style='font-size:40px'> Machine Learning with MLlib</h1>
<div> 
    <ul style='font-size:20px'> 
        <li> 
            Iremos aprender a utilizar a spark.ml com um dataset sobre aluguéis do Airbnb em São Francisco.  
        </li>
    </ul>
</div>

<h2 style='font-size:30px'> Data Ingestion and Exploration</h2>
<div> 
    <ul style='font-size:20px'> 
        <li> 
            Tive que remover casas com preço 0.00 dólares e preencher valores nulos com a mediana das colunas. &#x24;'s foram deletados das colunas de preço também.
        </li>
    </ul>
</div>

In [1]:
import pandas as pd
df = pd.read_csv('https://raw.githubusercontent.com/databricks/LearningSparkV2/master/databricks-datasets/learning-spark-v2/sf-airbnb/sf-airbnb.csv',
                usecols=['neighbourhood', 'room_type', 'bedrooms', 'bathrooms', 'number_of_reviews', 'price'])

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('C.10').getOrCreate()

22/06/18 20:09:00 WARN Utils: Your hostname, veiga-Inspiron resolves to a loopback address: 127.0.1.1; using 192.168.15.21 instead (on interface wlp7s0)
22/06/18 20:09:00 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/06/18 20:09:01 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
airbnbDF = spark.createDataFrame(df, schema='''`neighborhood` STRING, `room_type` STRING, `bathrooms` FLOAT, `bedrooms` FLOAT, `price` STRING, 
                            `number_of_reviews` INT''')

In [4]:
from pyspark.ml.feature import Imputer
import pyspark.sql.functions as F

num_cols = ['bathrooms', 'bedrooms', 'number_of_reviews']
airbnbDF = (airbnbDF.withColumn('price', F.substring('price', 2, 10).cast('float'))
        .where('price>0'))

# O objeto 'Imputer' retorna versões das colunas selecionadas com o devido preenchimento dos NaN'.
imputer = Imputer(strategy='median', inputCols=num_cols, outputCols=[f'{col}_na' for col in num_cols])
airbnbDF = imputer.fit(airbnbDF).transform(airbnbDF)
airbnbDF = airbnbDF.drop('bathrooms', 'bedrooms', 'number_of_reviews')
airbnbDF.show()

+--------------------+---------------+-----+------------+-----------+--------------------+
|        neighborhood|      room_type|price|bathrooms_na|bedrooms_na|number_of_reviews_na|
+--------------------+---------------+-----+------------+-----------+--------------------+
|     Duboce Triangle|Entire home/apt|170.0|         1.0|        1.0|                 180|
|      Bernal Heights|Entire home/apt|235.0|         1.0|        2.0|                 111|
|         Cole Valley|   Private room| 65.0|         4.0|        1.0|                  17|
|         Cole Valley|   Private room| 65.0|         4.0|        1.0|                   8|
|Western Addition/...|Entire home/apt|785.0|         1.5|        2.0|                  27|
|Western Addition/...|Entire home/apt|255.0|         1.0|        2.0|                  31|
|    Mission District|   Private room|139.0|         1.0|        1.0|                 647|
|        Potrero Hill|   Private room|135.0|         1.0|        1.0|                 453|

<h2 style='font-size:30px'> Creating Training and Test Sets</h2>
<div> 
    <ul style='font-size:20px'> 
        <li> 
            O autor ensina o funcionamento de um train-test-split e o implanta em "AirbnbDF".
        </li>
    </ul>
</div>

In [5]:
# Criação dos sets de treino e teste.
trainDF, testDF = airbnbDF.randomSplit([0.8, 0.2], seed=42)

<h2 style='font-size:30px'> Preparing Features with Transformers</h2>
<div> 
    <ul style='font-size:20px'> 
        <li> 
            O aprendizado dos algoritmos no Spark é um tanto diferente quando comparado aos do scikit-learn. Por aqui, os modelos exigem que todas as features de uma estejam contidas dentro de um vetor. Nesse sentido, é necessário criar nos datasets uma coluna adicional na qual serão adicionados os vetores.
        </li>
    </ul>
</div>

In [6]:
# Essa tarefa contará com o transformador 'VectorAssembler'.
from pyspark.ml.feature import VectorAssembler

vecAssembler = VectorAssembler(inputCols=['bedrooms_na'], outputCol='features')

# Aparentemente, o 'VectorAssembler' pode funcionar sem um 'fit'.
# Por ora, iremos usar apenas o número de quartos como variável independente.
vecTrainDF = vecAssembler.transform(trainDF)

vecTrainDF.show(5)

+------------+---------------+-----+------------+-----------+--------------------+--------+
|neighborhood|      room_type|price|bathrooms_na|bedrooms_na|number_of_reviews_na|features|
+------------+---------------+-----+------------+-----------+--------------------+--------+
|Alamo Square|Entire home/apt| 94.0|         1.0|        0.0|                  28|   [0.0]|
|Alamo Square|Entire home/apt|100.0|         1.0|        1.0|                  13|   [1.0]|
|Alamo Square|Entire home/apt|136.0|         1.5|        1.0|                 211|   [1.0]|
|Alamo Square|Entire home/apt|200.0|         1.5|        1.0|                   6|   [1.0]|
|Alamo Square|Entire home/apt|275.0|         1.0|        2.0|                  71|   [2.0]|
+------------+---------------+-----+------------+-----------+--------------------+--------+
only showing top 5 rows



<h2 style='font-size:30px'> Using Estimators to Build Models</h2>
<div> 
    <ul style='font-size:20px'> 
        <li> 
            Treinando uma regressão linear simples em nossos dados.
        </li>
    </ul>
</div>

In [7]:
from pyspark.ml.regression import LinearRegression
lr = LinearRegression(featuresCol='features', labelCol='price')

# LinearRegression.fit() nos retorna o transformador LinearRegressionModel, responsável por fazer previsões.
lrModel = lr.fit(vecTrainDF)

22/06/18 20:09:13 WARN Instrumentation: [e8ff97e2] regParam is zero, which might cause numerical instability and overfitting.
22/06/18 20:09:13 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
22/06/18 20:09:13 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.ForeignLinkerBLAS
22/06/18 20:09:13 WARN InstanceBuilder$NativeLAPACK: Failed to load implementation from:dev.ludovic.netlib.lapack.JNILAPACK


In [8]:
# Obtendo os coeficiente e o intercept do modelo.
lrModel.coefficients, lrModel.intercept

(DenseVector([88.4463]), 74.90401139425546)

In [9]:
# Interessante! Use 'explainParam' para obter uma breve descrição sobre a utilidade de um dado argumento.
lrModel.explainParam('epsilon')

'epsilon: The shape parameter to control the amount of robustness. Must be > 1.0. Only valid when loss is huber (default: 1.35)'

<h2 style='font-size:30px'> Creating a Pipeline</h2>
<div> 
    <ul style='font-size:20px'> 
        <li> 
            Assim como no scikit-learn, podemos criar Pipelines de transformação dos dados a fim de facilitar futuras manipulações.
        </li>
    </ul>
</div>

In [10]:
# Vamos criar uma Pipeline que transforme o dataset de teste. Afinal, ele tem que estar em condições semelhantes ao de treino
# para que as previsões sejam possíveis.
from pyspark.ml import Pipeline

# Assim como a Regressão Logística, 'fit' cria um PipelineModel.
# Adaptando a pipeline ao dataset de treino
pipelineModel = Pipeline(stages=[vecAssembler, lr]).fit(trainDF)

# As previsões do modelo são apresentadas como uma coluna no dataset. E observe: elas são calculadas com um 'transform', e não 'predict'.                                                                                                                                 
pipelineModel.transform(testDF).show(5)

22/06/18 20:09:15 WARN Instrumentation: [5aa69a5f] regParam is zero, which might cause numerical instability and overfitting.


+--------------+---------------+-----+------------+-----------+--------------------+--------+------------------+
|  neighborhood|      room_type|price|bathrooms_na|bedrooms_na|number_of_reviews_na|features|        prediction|
+--------------+---------------+-----+------------+-----------+--------------------+--------+------------------+
|  Alamo Square|Entire home/apt|100.0|         1.0|        1.0|                  65|   [1.0]|163.35027466456214|
|  Alamo Square|Entire home/apt|300.0|         2.0|        3.0|                  10|   [3.0]| 340.2428012051755|
|  Alamo Square|Entire home/apt|785.0|         1.5|        2.0|                   3|   [2.0]| 251.7965379348688|
|Balboa Terrace|Entire home/apt|125.0|         1.0|        2.0|                 321|   [2.0]| 251.7965379348688|
|Balboa Terrace|   Private room| 49.0|         3.0|        1.0|                  75|   [1.0]|163.35027466456214|
+--------------+---------------+-----+------------+-----------+--------------------+--------+---

<h3 style='font-size:30px;font-style:italic'> One-hot encoding</h3>


<p style='color:red'> Fazer a explicação do One-hot encoding